# 課題 銀行口座の管理

銀行口座の管理プログラムを作成しましょう。ユーザーが残高を直接変更できないように、カプセル化を使って残高を管理します。預け入れ（deposit）や引き出し（withdraw）が正常に行えるよう、残高不足の場合はエラーを表示する機能も実装します。

`05_oop` フォルダの中に `task01.ipynb` というノートブックを作成し、以下の仕様を満たすプログラムを記述してください。

## 仕様
### クラスの仕様：

`BankAccount` クラスを作成し、残高を管理するための属性を定義してください。

- 残高（balance）属性を定義し、初期残高を指定してインスタンス化できるようにします。残高属性は外部から直接アクセスできないようにしてください。
- 以下のメソッドを定義してください：
  - `deposit(amount)`: 預け入れメソッド。指定された金額を残高に追加します。預け入れる金額が0より小さい場合は  
    `エラー: 正しい金額を入力してください` と表示して処理を終了します。
  - `withdraw(amount)`: 引き出しメソッド。指定された金額を残高から引き出します。引き出し可能な場合のみ残高を減らし、残高不足や0以下の金額を指定した場合は  
    `エラー: 残高不足または無効な金額です` と表示して処理を終了します。
  - `get_balance()`: 現在の残高を返します。

### 呼び出し側の仕様：

以下の操作を順に行ってください。

- 初期残高1000円で `BankAccount` クラスのインスタンスを作成  
- 500円を預け入れ  
- 200円を引き出し  
- 現在の残高を表示（`1300` と表示される）  
- 1500円を引き出そうとする（`エラー: 残高不足または無効な金額です。`）

## 解き方のヒント

カプセル化により、残高（balance）属性は外部から直接アクセスできません。  
残高の確認には `get_balance()` メソッドを、残高の変更には `deposit()` と `withdraw()` メソッドを使用してください。

## 実行結果の例：

```python
1300
エラー: 残高不足または無効な金額です。
```

**預け入れ時に無効な金額（-500など）を指定した場合：**

```
エラー: 正しい金額を入力してください
```

## 回答

In [ ]:
class BankAccount:
  __balance: int

  def __init__(self, balance: int):
    self.__balance = balance

  def deposit(self, amount: int) -> None:
    if amount < 0:
      print("エラー: 正しい金額を入力してください")
      return
    self.__balance += amount

  def withdraw(self, amount: int) -> None:
    if not(0 < amount <= self.__balance):
      print("エラー: 残高不足または無効な金額です")
      return
    self.__balance -= amount

  def get_balance(self) -> int:
    return self.__balance

In [11]:
account = BankAccount(1000)
account.deposit(500)
account.withdraw(200)
print(account.get_balance())
account.withdraw(1500)

1300
エラー: 残高不足または無効な金額です


## テスト

In [74]:
from unittest import TestCase, TestLoader, TestSuite, TextTestRunner
from io import StringIO
import sys

class TestBankAccountBase(TestCase):
  def setUp(self):
    self.account = BankAccount(1000)
    self.io = StringIO()
    sys.stdout = self.io
    
  def tearDown(self):
    sys.stdout = sys.__stdout__

class TestBankAccountInit(TestBankAccountBase):
  def test_init(self):
    self.assertEqual(self.account.get_balance(), 1000)

class TestBankAccountDeposit(TestBankAccountBase):
  def test_deposit_valid(self):
    self.account.deposit(500)
    self.assertEqual(self.account.get_balance(), 1500)
    
  def test_deposit_valid_zero(self):
    self.account.deposit(0)
    self.assertEqual(self.account.get_balance(), 1000)
    
  def test_deposit_invalid_negative(self):
    self.account.deposit(-500)
    self.assertIn("エラー: 正しい金額を入力してください", self.io.getvalue())
    self.assertEqual(self.account.get_balance(), 1000)

class TestBankAccountWithdraw(TestBankAccountBase):
  def test_withdraw_valid(self):
    self.account.withdraw(200)
    self.assertEqual(self.account.get_balance(), 800)
    
  def test_withdraw_valid_exact_balance(self):
    self.account.withdraw(1000)
    self.assertEqual(self.account.get_balance(), 0)
    
  def test_withdraw_invalid_zero(self):
    self.account.withdraw(0)
    self.assertIn("エラー: 残高不足または無効な金額です", self.io.getvalue())
    self.assertEqual(self.account.get_balance(), 1000)
    
  def test_withdraw_invalid_negative(self):
    self.account.withdraw(-100)
    self.assertIn("エラー: 残高不足または無効な金額です", self.io.getvalue())
    self.assertEqual(self.account.get_balance(), 1000)
    
  def test_withdraw_invalid_insufficient_balance(self):
    self.account.withdraw(1500)
    self.assertIn("エラー: 残高不足または無効な金額です", self.io.getvalue())
    self.assertEqual(self.account.get_balance(), 1000)

class TestBankAccountScenario(TestBankAccountBase):
  def test_scenario_insufficient_balance(self):
    self.account.deposit(500)
    self.assertEqual(self.account.get_balance(), 1500)
    self.account.withdraw(200)
    self.assertEqual(self.account.get_balance(), 1300)
    self.account.withdraw(1500)
    self.assertIn("エラー: 残高不足または無効な金額です", self.io.getvalue())
    self.assertEqual(self.account.get_balance(), 1300)

if __name__ == '__main__':
  loader = TestLoader()
  suite = TestSuite([
    loader.loadTestsFromTestCase(TestBankAccountInit),
    loader.loadTestsFromTestCase(TestBankAccountDeposit),
    loader.loadTestsFromTestCase(TestBankAccountWithdraw),
    loader.loadTestsFromTestCase(TestBankAccountScenario)
  ])
  runner = TextTestRunner(verbosity=2)
  runner.run(suite)

test_init (__main__.TestBankAccountInit.test_init) ... ok
test_deposit_invalid_negative (__main__.TestBankAccountDeposit.test_deposit_invalid_negative) ... ok
test_deposit_valid (__main__.TestBankAccountDeposit.test_deposit_valid) ... ok
test_deposit_valid_zero (__main__.TestBankAccountDeposit.test_deposit_valid_zero) ... ok
test_withdraw_invalid_insufficient_balance (__main__.TestBankAccountWithdraw.test_withdraw_invalid_insufficient_balance) ... ok
test_withdraw_invalid_negative (__main__.TestBankAccountWithdraw.test_withdraw_invalid_negative) ... ok
test_withdraw_invalid_zero (__main__.TestBankAccountWithdraw.test_withdraw_invalid_zero) ... ok
test_withdraw_valid (__main__.TestBankAccountWithdraw.test_withdraw_valid) ... ok
test_withdraw_valid_exact_balance (__main__.TestBankAccountWithdraw.test_withdraw_valid_exact_balance) ... ok
test_scenario_insufficient_balance (__main__.TestBankAccountScenario.test_scenario_insufficient_balance) ... ok

----------------------------------------